
# IBM Applied Data Science Capstone Course by Coursera
## Week 5 project code
### Start your shopping mall in Kochi, Kerala, India

It consists of:

1. Build a dataframe of neighborhoods in Kochi, Kerala by web scraping the data from Wikipedia page
2. Get the geographical coordinates of the neighborhoods
3. Obtain the venue data for the neighborhoods from Foursquare API
4. Explore and cluster the neighborhoods
5. Select the best cluster to open a new shopping mall

In [3]:
pip install geocoder

     |████████████████████████████████| 98 kB 5.7 MB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [5]:
!pip install beautifulsoup4
!pip install lxml

!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: / 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
                                                                                                  |failed

UnsatisfiableError: The following specifications were found
to be incompatible with the existing python installation in your environment:

Specifications:

  - cffi -> python[version='2.7.*|3.5.*|3.6.*|3.6.12|>=3.6,<3.7.0a0|>=3.7,<3.8.0a0|>=3.9,<3.10.0a0|>=3.8,<3.9.0a0|3.6.9|3.6.9|3.6.9|>=2.7,<2.8.0a0|3.6.9|>=3.5,<3.6.0a0|3.4.*',build='1_73_pypy|2_73_pypy|3_73_pypy|4_73_pypy|0_73_pypy']
  - rsa -> python[version='2.7.*|3.4.*|3.5.*|3.6.*']

Your python: python=3.7

If python is on the left-most side of the chain, that's the version you've asked for.
When python appears to the right, tha

In [8]:
pip install folium


     |████████████████████████████████| 93 kB 3.0 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [9]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


In [10]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_of_Kochi").text

In [11]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [12]:
# create a list to store neighborhood data
neighborhoodList = []

In [13]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [14]:
# create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

kl_df.head()

,Neighborhood
0,Alangad
1,Angamaly
2,Aroor
3,Chellanam
4,Chendamangalam


In [15]:
# print the number of rows of the dataframe
kl_df.shape

(44, 1)

3. Get the geographical coordinates

In [16]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Kochi, Kerala'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [17]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [18]:
coords

[[10.847500000000025, 76.43609000000004],
 [10.20366000000007, 76.38268000000005],
 [9.936010000000067, 76.26142000000004],
 [9.835260000000062, 76.27029000000005],
 [10.172920000000033, 76.23346000000004],
 [10.15354000000002, 76.34068000000008],
 [10.060850000000073, 76.28901000000008],
 [9.96118000000007, 76.30659000000009],
 [10.081320000000062, 76.34155000000004],
 [9.939106429926778, 76.3750355385582],
 [10.128150000000062, 76.37217000000004],
 [10.086410000000058, 76.38181000000003],
 [9.957580000000064, 76.24239000000006],
 [9.966870000000029, 76.35720000000003],
 [10.06352000000004, 76.24660000000006],
 [9.988446881019321, 76.30342723952937],
 [9.947600000000023, 76.26079000000004],
 [10.107690000000048, 76.26171000000005],
 [10.055610000000058, 76.27164000000005],
 [10.103150000000028, 76.24615000000006],
 [9.90220000000005, 76.31064000000003],
 [9.940510000000074, 76.32395000000008],
 [9.952060000000074, 76.25080000000008],
 [9.999140000000068, 76.26241000000005],
 [9.930700

In [19]:

# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df

(44, 3)


,Neighborhood,Latitude,Longitude
0,Alangad,10.847500,76.436090
1,Angamaly,10.203660,76.382680
2,Aroor,9.936010,76.261420
3,Chellanam,9.835260,76.270290
4,Chendamangalam,10.172920,76.233460
5,"Chengamanad, Ernakulam district",10.153540,76.340680
6,Cheranallur,10.060850,76.289010
7,Chilavannoor,9.961180,76.306590
8,Choornikkara,10.081320,76.341550
9,Chottanikkara,9.939106,76.375036


In [21]:
# save the DataFrame as CSV file
kl_df.to_csv("kl_df.csv", index=False)

4. Create a map of Kochi with neighborhoods superimposed on top

In [24]:
# get the coordinates of Kochi
address = 'Kochi, Kerala'

geolocator = Nominatim(user_agent="myapplication")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Kochi, Kerala {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Kochi, Kerala 9.9313695, 76.2673759.


In [25]:
# create map of Kochi using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [26]:
# save the map as HTML file
map_kl.save('map_kl.html')

5. Use the Foursquare API to explore the neighborhoods

In [28]:
# define Foursquare Credentials and Version
CLIENT_ID = 'PHZ0EDYCPD3XEDKLSLRCBYMNJL4XAIWOROZ01YRX0ZMMGF3B' # your Foursquare ID
CLIENT_SECRET = 'IHS4T5KXCVNMLVR0RK3DX3O4UUIXYLIP52CAYNK2U1GAKZRO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: PHZ0EDYCPD3XEDKLSLRCBYMNJL4XAIWOROZ01YRX0ZMMGF3B
CLIENT_SECRET:IHS4T5KXCVNMLVR0RK3DX3O4UUIXYLIP52CAYNK2U1GAKZRO


In [29]:
# Now, let's get the top 100 venues that are within a radius of 2000 meters.
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [30]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(808, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Angamaly,10.20366,76.38268,Carnival Cinemas,10.195147,76.386157,Multiplex
1,Angamaly,10.20366,76.38268,Carnival Cinemas Multiplex,10.195266,76.386193,Multiplex
2,Angamaly,10.20366,76.38268,Angamally Bus Stand,10.196622,76.385227,Bus Station
3,Angamaly,10.20366,76.38268,Saravana Bhavan,10.195313,76.384040,Indian Restaurant
4,Angamaly,10.20366,76.38268,Elite Palazzo,10.189762,76.386080,Hotel


In [31]:
venues_df.groupby(["Neighborhood"]).count()


,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Angamaly,5,5,5,5,5,5
Aroor,14,14,14,14,14,14
Chendamangalam,5,5,5,5,5,5
"Chengamanad, Ernakulam district",4,4,4,4,4,4
Cheranallur,4,4,4,4,4,4
Chilavannoor,64,64,64,64,64,64
Choornikkara,6,6,6,6,6,6
Chottanikkara,6,6,6,6,6,6
Chowwara,2,2,2,2,2,2


In [32]:
# Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))


There are 125 uniques categories.


In [35]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:125]

array(['Multiplex', 'Bus Station', 'Indian Restaurant', 'Hotel',
       'Fried Chicken Joint', 'Indie Movie Theater', 'Hotel Bar',
       'Restaurant', 'Gym', 'Halal Restaurant', 'Fish Market',
       'Department Store', 'Smoke Shop', 'Fast Food Restaurant',
       'Tea Room', 'Light Rail Station', 'Auditorium', 'Asian Restaurant',
       'Historic Site', 'River', 'Boat or Ferry', 'Astrologer',
       'Comfort Food Restaurant', 'Bakery', 'Food Truck',
       'South Indian Restaurant', 'Café', 'Thai Restaurant',
       'French Restaurant', 'Nightclub', 'Stadium', 'Athletics & Sports',
       'Sandwich Place', 'Ice Cream Shop', 'Chinese Restaurant',
       'Motorcycle Shop', 'Burger Joint', 'Donut Shop', 'Park',
       'Gym / Fitness Center', 'Juice Bar', 'Pizza Place',
       'Middle Eastern Restaurant', 'Clothing Store', 'Coffee Shop',
       'Vegetarian / Vegan Restaurant', 'Dhaba', 'Bar', 'Lounge',
       "Men's Store", 'Shoe Store', 'Food', 'Furniture / Home Store',
       'Snack Pl

In [34]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

6. Analyze Each Neighborhood

In [36]:

# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(808, 126)


,Neighborhoods,ATM,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,Arcade,Arepa Restaurant,Art Gallery,Asian Restaurant,Astrologer,Athletics & Sports,Auditorium,Auto Garage,BBQ Joint,Bakery,Bank,Bar,Beach,Bed & Breakfast,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Burger Joint,Bus Station,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Department Store,Dessert Shop,Dhaba,Diner,Donut Shop,Electronics Store,Event Space,Fast Food Restaurant,Fish Market,Fishing Store,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gastropub,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Kerala Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Middle Eastern Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Neighborhood,New American Restaurant,Nightclub,Park,Performing Arts Venue,Pizza Place,Playground,Pool,Portuguese Restaurant,Punjabi Restaurant,Recreation Center,Resort,Restaurant,River,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Club,Stadium,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Train Station,Vegetarian / Vegan Restaurant
0,Angamaly,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Angamaly,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Angamaly,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Angamaly,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Angamaly,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [37]:
# Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(42, 126)


,Neighborhoods,ATM,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,Arcade,Arepa Restaurant,Art Gallery,Asian Restaurant,Astrologer,Athletics & Sports,Auditorium,Auto Garage,BBQ Joint,Bakery,Bank,Bar,Beach,Bed & Breakfast,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Burger Joint,Bus Station,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Department Store,Dessert Shop,Dhaba,Diner,Donut Shop,Electronics Store,Event Space,Fast Food Restaurant,Fish Market,Fishing Store,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gastropub,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Kerala Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Middle Eastern Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Neighborhood,New American Restaurant,Nightclub,Park,Performing Arts Venue,Pizza Place,Playground,Pool,Portuguese Restaurant,Punjabi Restaurant,Recreation Center,Resort,Restaurant,River,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Club,Stadium,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Train Station,Vegetarian / Vegan Restaurant
0,Angamaly,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.400000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Aroor,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.071429,0.0,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.071429,0.000000,0.0,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000
2,Chendamangalam,0.000000,0.00

In [38]:
len(kl_grouped[kl_grouped["Shopping Mall"] > 0])


6

Create a new DataFrame for Shopping Mall data only

In [39]:
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]

kl_mall.head()

,Neighborhoods,Shopping Mall
0,Angamaly,0.0
1,Aroor,0.0
2,Chendamangalam,0.0
3,"Chengamanad, Ernakulam district",0.0
4,Cheranallur,0.0


7. Cluster Neighborhoods
Run k-means to cluster the neighborhoods in Kochi into 3 clusters.

In [58]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [59]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [60]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Angamaly,0.0,0
1,Aroor,0.0,0
2,Chendamangalam,0.0,0
3,"Chengamanad, Ernakulam district",0.0,0
4,Cheranallur,0.0,0


In [61]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(42, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Angamaly,0.0,0,10.20366,76.38268
1,Aroor,0.0,0,9.93601,76.26142
2,Chendamangalam,0.0,0,10.17292,76.23346
3,"Chengamanad, Ernakulam district",0.0,0,10.15354,76.34068
4,Cheranallur,0.0,0,10.06085,76.28901


In [62]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(42, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Angamaly,0.000000,0,10.203660,76.382680
21,Mulavukad,0.000000,0,9.999140,76.262410
22,Mundamveli,0.000000,0,9.930700,76.253200
23,Nedumbassery,0.000000,0,10.156690,76.377800
24,Nettoor,0.000000,0,9.927260,76.311810
25,Pachalam,0.000000,0,10.003470,76.281220
26,Palluruthy,0.000000,0,9.916420,76.275670
27,Pathalam,0.000000,0,10.078170,76.318570
29,Thevara,0.000000,0,9.942090,76.298390
30,Thiruvankulam,0.000000,0,9.946350,76.367460


Finally, let's visualize the resulting clusters

In [63]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [64]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

8. Examine Clusters
Cluster 0

In [65]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Angamaly,0.0,0,10.203660,76.382680
21,Mulavukad,0.0,0,9.999140,76.262410
22,Mundamveli,0.0,0,9.930700,76.253200
23,Nedumbassery,0.0,0,10.156690,76.377800
24,Nettoor,0.0,0,9.927260,76.311810
25,Pachalam,0.0,0,10.003470,76.281220
26,Palluruthy,0.0,0,9.916420,76.275670
27,Pathalam,0.0,0,10.078170,76.318570
29,Thevara,0.0,0,9.942090,76.298390
30,Thiruvankulam,0.0,0,9.946350,76.367460


Cluster 1

In [66]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
33,Thrippunithura,0.058824,1,9.94111,76.34698
19,Maradu,0.043478,1,9.94051,76.32395


Cluster 2

In [67]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
31,Thrikkakkara,0.022472,2,10.017360,76.316370
13,Karanakodam,0.014286,2,9.988447,76.303427
39,Vennala,0.029412,2,9.995380,76.322430
28,Thammanam,0.021739,2,9.985570,76.311300


### Observations and inference:

+ Kochi is a place with its fair number of shopping destinations for people. It can be seen that shopping malls are concentrated in cluster 1 with almost as many shopping malls as cluster 1 in cluster 2 and none in cluster 0. 
+ This shows that there is a lot of potential for shopping malls in areas that belong to cluster 0 because of the lack of competition and a need for a common destination point with restaurants, banks, theatres, professional offices, service stations, and other establishments. The area required to build a shopping mall can also be acquired at a low cost in cluster 0 and hence it would be easier to break even and start getting profits. 
+ If a shopping mall is brought about in cluster 1 or cluster 2, it would suffer from the intence competition between the already existing shopping outlets and the upcoming ones. Therefore, this project can be utilized by project architects to capitalize on these observations in building shopping malls in different neighbourhoods of Kochi.

